<a href="https://colab.research.google.com/github/fehimecapar/ProjectRAG/blob/main/ProjectRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') #Drive connection due to access log file

Mounted at /content/drive


In [2]:

log_file_path = '/content/drive/MyDrive/archive/access.log'

# access.log file reading
with open(log_file_path, 'r') as file:
    logs = file.readlines()

# show the first few data
for line in logs[:10]:
    print(line)

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/1.1" 200 16

In [3]:
import pandas as pd

# I will hold the loge data as a list
log_data = []

for line in logs:
    parts = line.split()
    ip = parts[0]
    timestamp = parts[3] + ' ' + parts[4]
    request = parts[5] + ' ' + parts[6] + ' ' + parts[7]
    status = parts[8]
    size = parts[9]

    log_data.append([ip, timestamp, request, status, size])

# convert the data to pandas DataFrame
log_df = pd.DataFrame(log_data, columns=['IP', 'Timestamp', 'Request', 'Status', 'Size'])

# show the first few data
print(log_df.head())

              IP                     Timestamp  \
0   54.36.149.41  [22/Jan/2019:03:56:14 +0330]   
1    31.56.96.51  [22/Jan/2019:03:56:16 +0330]   
2    31.56.96.51  [22/Jan/2019:03:56:16 +0330]   
3  40.77.167.129  [22/Jan/2019:03:56:17 +0330]   
4    91.99.72.15  [22/Jan/2019:03:56:17 +0330]   

                                             Request Status   Size  
0  "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%...    200  30577  
1   "GET /image/60844/productModel/200x200 HTTP/1.1"    200   5667  
2   "GET /image/61474/productModel/200x200 HTTP/1.1"    200   5379  
3   "GET /image/14925/productModel/100x100 HTTP/1.1"    200   1696  
4  "GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D...    200  41483  


In [4]:
# remove IP, Timestamp and Size rows
columns_to_drop = ['IP', 'Timestamp', 'Size']

# this rows were from the DataFrame removed
log_df = log_df.drop(columns=columns_to_drop)

# show updated Data Frame
print(log_df.head())

                                             Request Status
0  "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%...    200
1   "GET /image/60844/productModel/200x200 HTTP/1.1"    200
2   "GET /image/61474/productModel/200x200 HTTP/1.1"    200
3   "GET /image/14925/productModel/100x100 HTTP/1.1"    200
4  "GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D...    200


In [5]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
from sentence_transformers import SentenceTransformer

# upload the Sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Convert query data to vectors
#request_vectors = model.encode(log_df['Request'].tolist())


request_vectors = model.encode(log_df['Request'].tolist()[:20000])


"""
batch_size = 1000
request_vectors = []

for i in range(0, len(log_df['Request']), batch_size):
    batch_vectors = model.encode(log_df['Request'].tolist()[i:i+batch_size])
    request_vectors.extend(batch_vectors)"""

# show
print(request_vectors [:5])

[[-0.3913801   0.40831926 -0.32646573 ... -0.12227561 -0.45891532
   0.12487593]
 [-0.50558215  0.23135157  0.15166956 ... -0.01784453  0.09444963
   0.11009549]
 [-0.5128455   0.23854263  0.04412114 ... -0.10354773  0.08211669
   0.11804578]
 [-0.58550334  0.5079506   0.09543234 ... -0.06599379  0.25848654
  -0.0630791 ]
 [-0.5286532   0.48364165 -0.4632836  ... -0.14491487 -0.08093496
   0.01941908]]


In [8]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.9 MB/s eta 0:00:00


In [9]:
import faiss
import numpy as np

# define vector size
d = request_vectors.shape[1]

# FAISS index
index = faiss.IndexFlatL2(d)

# add the vectors to FAISS index
index.add(np.array(request_vectors))

# show info about FAISS index
print(f"Toplam eklendi: {index.ntotal}")

Toplam eklendi: 20000


In [10]:
def find_similar_request(query):
    # vectorize the query
    query_vector = model.encode([query])

    # find the knn(k nearest neighboor)
    D, I = index.search(query_vector, k=5)

    # show the nearest results
    return log_df.iloc[I[0]]

# query
query = "How many times was the home page accessed?"
result = find_similar_request(query)
print(result)

                                                 Request Status
14586      "GET /blog/home-appliances/page/34/ HTTP/1.1"    200
5653               "GET /browse/Paper%20Shreds HTTP/1.1"    301
8353       "GET /blog/home-appliances/page/93/ HTTP/1.1"    200
3623   "GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9...    200
3660   "GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%...    200


In [11]:
# input
user_query = "How many times was the home page accessed?"

# vectorize query
query_vector = model.encode([user_query])

# Finding the closest log records to the query (z.b. first 5 closest records)
D, I = index.search(query_vector, k=5)

# Get the closest log records found
retrieved_logs = log_df.iloc[I[0]]

# View found records
print(retrieved_logs)

                                                 Request Status
14586      "GET /blog/home-appliances/page/34/ HTTP/1.1"    200
5653               "GET /browse/Paper%20Shreds HTTP/1.1"    301
8353       "GET /blog/home-appliances/page/93/ HTTP/1.1"    200
3623   "GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9...    200
3660   "GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%...    200


In [12]:
D, I = index.search(query_vector, k=5)  # returns the first 5 closest results

# Get the closest log records found
retrieved_logs = log_df.iloc[I[0]]
# show
print(retrieved_logs)

                                                 Request Status
14586      "GET /blog/home-appliances/page/34/ HTTP/1.1"    200
5653               "GET /browse/Paper%20Shreds HTTP/1.1"    301
8353       "GET /blog/home-appliances/page/93/ HTTP/1.1"    200
3623   "GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9...    200
3660   "GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%...    200


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Install GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
generation_model = GPT2LMHeadModel.from_pretrained("gpt2")
generation_model.config.pad_token_id = generation_model.config.eos_token_id

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
# Combine the found log records into a single string
context = "\n".join(retrieved_logs['Request'].tolist())

# Combine the query and logs into a format that the model can use as input
input_text = f"Context: {context}\nQuestion: {user_query}\nAnswer:"

# Tokenize
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [15]:
# create an answer using the model
output = generation_model.generate(input_ids, max_length=250, num_return_sequences=1)

# Parse and print the generated response
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Context: "GET /blog/home-appliances/page/34/ HTTP/1.1"
"GET /browse/Paper%20Shreds HTTP/1.1"
"GET /blog/home-appliances/page/93/ HTTP/1.1"
"GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86 HTTP/1.1"
"GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C HTTP/1.1"
Question: How many times was the home page accessed?
Answer: The home page was accessed by a user of the website.
Question: How many times was the home page accessed by a user of the website? Answer: The home page was accessed by a user of


In [24]:

def generate_answer(user_query):
    # info (retrieval)
    query_vector = model.encode([user_query])
    D, I = index.search(query_vector, k=5)

    if len(I[0]) == 0:
        return "No relevant logs found."

    # merge the found log records
    retrieved_logs = log_df.iloc[I[0]]
    context = "\n".join(retrieved_logs['Request'].tolist())
    input_text = f"Context: {context}\nQuestion: {user_query}\nAnswer:"

    # generation
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # mask
    attention_mask = (input_ids != generation_model.config.pad_token_id).long()

    output = generation_model.generate(input_ids, attention_mask=attention_mask,
                                       max_length=250, num_return_sequences=1,
                                       no_repeat_ngram_size=2)  # Tekrarlı ngram'leri engelleme

    # response
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

# input
user_query = "who accessed to the home page and which link from?"
print(generate_answer(user_query))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: "GET /blog/home-appliances/page/34/ HTTP/1.1"
"GET /browse/home-theater-system HTTP/1.1"
"GET /browse/home-theater-system HTTP/1.1"
"GET /blog/home-appliances/page/93/ HTTP/1.1"
"GET /image/get?path=/Image/757094_574.jpg HTTP/1.1"
Question: who accessed to the home page and which link from?
Answer: The user who requested the URL.
The user requesting the url. The URL is the same as the one that was requested. If the user requested a different URL, the request was made to a third party. This is called a "requested URL".
If the User has requested an URL from the Home Page, then the Requested URI is used. However, if the URI was not provided, it is not used, and the requested URL was provided to another user. In this case, a request is made from a user to an external site. For example, you might want to request a URL to your


In [33]:
# check dimensions
#print("vector dimension", query_vector.shape, "faiss index dimension", index.d)
"""
generate_answer(user_query) fonksiyonundan istediğim gibi bir sonuç alamayınca boyut kontrolü yapmak istedim
Stack oveerflow'da bulduğum bir kontrol satırı. Boyutlar uyumsuz olunca sonuçtan uzaklaşıldığını öğrendim
"""

'\ngenerate_answer(user_query) fonksiyonundan istediğim gibi bir sonuç alamayınca boyut kontrolü yapmak istedim\n'

In [26]:
user_query = "how many time was the home page accessed?"
print(generate_answer(user_query))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: "GET /blog/home-appliances/page/34/ HTTP/1.1"
"GET /blog/home-appliances/page/93/ HTTP/1.1"
"GET /browse/Paper%20Shreds HTTP/1.1"
"GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C HTTP/1.1"
"GET /m/event/Deawoo-Home-Promotion?page=1 HTTP/1.1"
Question: how many time was the home page accessed?
Answer: 1.5 seconds.
Example:
http://www.home.com/blog.php?id=34&page_id=-34
The home pages are accessed by the user. The user can choose to view the page or not. If the visitor is not logged in, the pages will be deleted. However


In [27]:
user_query = "how many times was the home page accessed?"
print(generate_answer(user_query))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: "GET /blog/home-appliances/page/34/ HTTP/1.1"
"GET /browse/Paper%20Shreds HTTP/1.1"
"GET /blog/home-appliances/page/93/ HTTP/1.1"
"GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86 HTTP/1.1"
"GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C HTTP/1.1"
Question: how many times was the home page accessed?
Answer: The home pages were accessed by the user.
The homepages were also accessed in the form of a search query. The search queries were:
http://www.homepages.com/search/


In [29]:
user_query = "how many times was the home page accessed?"
print(generate_answer(user_query))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: "GET /blog/home-appliances/page/34/ HTTP/1.1"
"GET /browse/Paper%20Shreds HTTP/1.1"
"GET /blog/home-appliances/page/93/ HTTP/1.1"
"GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86 HTTP/1.1"
"GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C HTTP/1.1"
Question: how many times was the home page accessed?
Answer: The home pages were accessed by the user.
The homepages were also accessed in the form of a search query. The search queries were:
http://www.homepages.com/search/


In [34]:
# test inputs
queries = [
    "How many times was the home page accessed?",
    "What was the most accessed page last month?",
    "Did any errors occur on the server?",
]

for query in queries:
    print(f"Question: {query}")
    print(f"Answer: {generate_answer(query)}")
    print("-" * 50)

Question: How many times was the home page accessed?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Context: "GET /blog/home-appliances/page/34/ HTTP/1.1"
"GET /browse/Paper%20Shreds HTTP/1.1"
"GET /blog/home-appliances/page/93/ HTTP/1.1"
"GET /m/browse/furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86 HTTP/1.1"
"GET /m/browse/Home-Furniture/%D9%85%D8%A8%D9%84%D9%85%D8%A7%D9%86-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C HTTP/1.1"
Question: How many times was the home page accessed?
Answer: The home pages were accessed by the user.
The homepages were also accessed in the form of a search query. The search queries were:
http://www.homepages.com/search/
--------------------------------------------------
Question: What was the most accessed page last month?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Context: "GET /browse/Paper%20Shreds HTTP/1.1"
"GET /browse/cover%20and%20bag%20for%20tablet HTTP/1.1"
"GET /blog/1392/09/page/4/ HTTP/1.1"
"GET /article/list/5665?page=0 HTTP/1.1"
"GET /article/list?page=908 HTTP/1.1"
Question: What was the most accessed page last month?
Answer: The most visited page was "The Daily Mail" on the day of the election.
The most viewed page is "Daily Mail".
What was your most read page? The Daily Mirror's most-read page, which was on Sunday, was:
1) "A Daily Express" (1,000,00)
2) The Guardian's (2,500,01) and the Daily Telegraph's, (3,200,02) (4,400,03)
--------------------------------------------------
Question: Did any errors occur on the server?
Answer: Context: "GET /errors/handle?f=p29%2Cb36%2Cb3+%D8%A8%D9%86%D9%81%D8%B4.%2Cb103 HTTP/1.1"
"GET /filter/b186%2Cp10280%2Cstexists HTTP/1.1"
"GET /filter/p65%2Crf150000000%2Cstexists%2Cb67 HTTP/1.1"
"GET /filter/p6%2Crf30000000%2Crt40000000%2Cstexists HTTP/1.1"
"GET /filter/b218%2Cp5666%2Cstexists H